In [10]:
import os
import gc
import sys
import cv2
import json
import glob
import torch
import random
import rasterio
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from collections import Counter
from rasterio.windows import Window
from matplotlib import pyplot as plt
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import directed_hausdorff
from torch.utils.data import DataLoader, Dataset

In [11]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m

    return img.reshape(shape).T

In [12]:
def dice_scores_img(pred, truth, eps=1e-8):
    """
    Dice metric for a single image as array.

    Args:
        pred (np array): Predictions.
        truth (np array): Ground truths.
        eps (float, optional): epsilon to avoid dividing by 0. Defaults to 1e-8.

    Returns:
        np array : dice value for each class
    """
    pred = pred.reshape(-1) > 0
    truth = truth.reshape(-1) > 0
    intersect = (pred & truth).sum(-1)
    union = pred.sum(-1) + truth.sum(-1)

    dice = (2.0 * intersect + eps) / (union + eps)
    return dice

In [13]:
DATA_PATH = r'C:/Users/soodn/Downloads/Naveksha/Kaggle HuBMAP/'
df = pd.read_csv('submission-deeplive.csv')
rles = pd.read_csv(DATA_PATH + 'Data/hubmap-kidney-segmentation-data/test.csv')
df_info = pd.read_csv(DATA_PATH + 'Data/hubmap-kidney-segmentation-data/HuBMAP-20-dataset_information.csv')

In [14]:
path_test = r'C:\Users\soodn\Downloads\Naveksha\Kaggle HuBMAP\Data\hubmap-kidney-segmentation-data\test'

In [15]:
rles, df

(          id                                           encoding
 0  2ec3f1bb9  60954223 34 60978180 69 61002168 74 61026156 7...
 1  3589adb90  68629509 19 68658940 53 68688371 72 68717803 7...
 2  d488c759a  534577255 4 534577263 5 534623911 19 534670566...
 3  aa05346ff  52826001 14 52856696 41 52887402 59 52918120 6...
 4  57512b7f1  329085519 25 329118756 31 329151993 37 3291852...,
           id                                          predicted
 0  2ec3f1bb9  60762303 28 60786293 28 60810269 54 60834259 5...
 1  3589adb90  68658983 26 68688416 26 68717813 70 68747246 7...
 2  d488c759a  537656743 8 537703403 8 537750059 14 537796719...
 3  aa05346ff  52918127 38 52948847 38 52979557 58 53010277 5...
 4  57512b7f1  329085513 30 329118753 30 329151987 50 3291852...)

In [16]:
def perf_metrics(gt, pred):
    n = 0
    d = 0
    for i in range(gt.shape[0]):
        for j in range (gt.shape[1]):
            if (gt[i][j]==pred[i][j]):
                n = n+1
            d = d+1
    
    return n/d, jaccard_score(gt.flatten(order='C'), pred.flatten(order='C')), directed_hausdorff(gt, pred)

In [17]:
df

,id,predicted
0,2ec3f1bb9,60762303 28 60786293 28 60810269 54 60834259 5...
1,3589adb90,68658983 26 68688416 26 68717813 70 68747246 7...
2,d488c759a,537656743 8 537703403 8 537750059 14 537796719...
3,aa05346ff,52918127 38 52948847 38 52979557 58 53010277 5...
4,57512b7f1,329085513 30 329118753 30 329151987 50 3291852...


In [18]:
DEBUG = False
sum_score = 0
sum_pa = 0
sum_ji = 0
sum_haus = 0

for img in rles['id'].unique():
    shape = df_info[df_info.image_file == img + ".tiff"][['width_pixels', 'height_pixels']].values.astype(int)[0]
    truth = rles[rles['id'] == img]['encoding']
    mask_truth = enc2mask(truth, shape)
    pred = df[df['id'] == img]['predicted']
    mask_pred = enc2mask(pred, shape)
    score = dice_scores_img(mask_pred, mask_truth)
    pa, ji, haus = perf_metrics(mask_pred, mask_truth)    
    sum_score += score
    sum_pa += pa
    sum_ji += ji
    sum_haus += haus[0]

In [19]:
sum_pa/5, sum_ji/5, sum_haus/5, sum_score/5

(0.9978311925238129,
 0.8651603636718331,
 23.034991353466427,
 0.9258956636996538)